In [36]:
import faiss
import os

# 构造 FAISS 向量数据库的路径
current_dir = os.getcwd()
faiss_index_path = os.path.join(current_dir, "..", "data", "vec_databases", "vecdb.faiss")

# 加载本地 FAISS 向量数据库
vec_db = faiss.IndexIDMap(faiss.IndexFlatIP(512))



In [33]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
import torch
from PIL import Image
import sys
sys.path.append(os.path.join(current_dir, ".."))
from src.outer_lab.facenet_pytorch import InceptionResnetV1

# Load YOLOv8 face detection model
face_detector = YOLO("train_model/train2/weights/best.pt")

# Load face feature extractor (assuming you're using a model like FaceNet or similar)
# This is a placeholder - replace with your actual feature extraction model
feature_extractor = InceptionResnetV1(pretrained='vggface2').eval()

# Function to process image and find matches
def process_image_and_find_matches(image_path, top_k=5):
    # Load and process the image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Failed to load image: {image_path}")
        return []
    
    # Detect faces using YOLOv8
    results = face_detector(img)
    
    matches = []
    
    # Process each detected face
    for result in results:
        boxes = result.boxes
        for box in boxes:
            # Get face bounding box coordinates
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            
            # Extract the face
            face = img[y1:y2, x1:x2]
            
            # Preprocess face for feature extraction
            face_pil = Image.fromarray(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))
            face_pil = face_pil.resize((160, 160))  # Adjust size according to your feature extractor
            face_tensor = torch.from_numpy(np.array(face_pil)).permute(2, 0, 1).float().unsqueeze(0) / 255.0
            
            # Extract face features
            with torch.no_grad():
                face_embedding = feature_extractor(face_tensor)
                face_embedding = face_embedding.cpu().numpy()
                face_embedding = face_embedding / np.linalg.norm(face_embedding)  # Normalize the embedding
                
            return face_embedding


In [34]:
image_path = "../data/photos/face1.png"
feature = process_image_and_find_matches(image_path)



0: 544x640 1 face, 17.8ms
Speed: 7.5ms preprocess, 17.8ms inference, 4.5ms postprocess per image at shape (1, 3, 544, 640)


In [35]:
type(feature)
res = index.search(feature, 5)
res

(array([[   0.073315,    0.025664, -3.4028e+38, -3.4028e+38, -3.4028e+38]], dtype=float32),
 array([[2008, 2007,   -1,   -1,   -1]], dtype=int64))